<a href="https://colab.research.google.com/github/megmarv/PsychoAI-/blob/Image-Retrieval/Custom_Made_Face_Recognition_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install required libraries
!pip install scikit-learn tensorflow opencv-python matplotlib deeplake

# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import deeplake

# Load Dataset from DeepLake
### Change the dataset path below ###
ds = deeplake.query('SELECT * FROM "hub://activeloop/wider-train"')
print(ds.summary())


Dataset length: 25275
Columns:
  images: kind=image, dtype=array(dtype=uint8, shape=(None, None, None))
  labels: kind=class_label, dtype=array(dtype=uint32, shape=(1))


None


In [ ]:
# Ensure that images are resized to the same size (e.g., 128x128)
IMG_SIZE = (128, 128)

# Extract and normalize images and labels
X = []
y = []

for sample in ds:
    try:
        img = sample['images']

        # Check if the image is 2D (grayscale) and expand dimensions if needed
        if len(img.shape) == 2:
            img = np.expand_dims(img, axis=-1)

        # Resize the image to a consistent shape (e.g., 128x128)
        img = load_img(img, target_size=IMG_SIZE)  # Resize to 128x128
        img = img_to_array(img)  # Convert to NumPy array

        # Normalize the image and append to the list
        X.append(img.astype('float32') / 255.0)

        # Append the label
        y.append(sample['labels'][0])

    except Exception as e:
        print(f"Error processing image: {e}")
        continue  # Skip this sample if there's an issue

# Convert the lists to NumPy arrays
X = np.array(X)
y = np.array(y)

# Check the shape of the final arrays
print(f"Final X shape: {X.shape}")
print(f"Final y shape: {y.shape}")

# Split dataset into training and testing (keep only 10,000 samples)
X, _, y, _ = train_test_split(X, y, train_size=10000, random_state=42)

# Split into training and testing sets (75% training, 25% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# Fit the generator on the training data
datagen.fit(X_train)

# Use flow method for data augmentation and memory efficiency
train_generator = datagen.flow(X_train, y_train, batch_size=32)
validation_generator = datagen.flow(X_test, y_test, batch_size=32)

# fitting the model with data generators
# model.fit(train_generator, validation_data=validation_generator, epochs=10, workers=4, use_multiprocessing=True)

# If you want to check the shape of the data after augmentation:
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")


In [ ]:
# Build CNN Model
def build_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Compile the Model
input_shape = X_train.shape[1:]
num_classes = len(np.unique(y))
model = build_model(input_shape, num_classes)
optimizer = SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the Model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=50,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping, reduce_lr])

# Model Evaluation
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)

# Predict on test data
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

# Metrics Evaluation
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Save the Model
model.save("face_recognition_model.h5")

def upload_and_test_model(model):
    uploaded = files.upload()
    for filename in uploaded.keys():
        img = cv2.imdecode(np.frombuffer(uploaded[filename], np.uint8), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (64, 64))
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=(0, -1))  # Add batch and channel dimensions

        pred = model.predict(img)
        pred_label = np.argmax(pred, axis=1)
        plt.imshow(img.squeeze(), cmap='gray')
        plt.title(f"Predicted Label: {pred_label[0]}")
        plt.show()